In [15]:
import os
import pandas as pd
import numpy as np

DIR = "./data/Raw/logRaws/"

NUM_SAMPLES = 2662
NUM_FEATURES = 6
NUM_FRAMES = 1000

file_names = os.listdir(DIR)
input_array = np.empty((NUM_SAMPLES, NUM_FRAMES, NUM_FEATURES))
output_array = np.empty((NUM_SAMPLES, NUM_FRAMES, 1))

# prepare data
def read_files(DIR):
    all_files = os.listdir(DIR)

    input_data = np.empty((0, 6), float)
    output_data = np.empty((0, 1), int)

    for file in all_files:
        file_path = os.path.join(DIR, file)
        df = pd.read_csv(file_path, usecols=[1, 3, 19, 20, 21, 25, 26, 27], header=None)
        if df.shape[0] < 999:
            continue

        df.columns = ['frame', 'status', 'accel_x', 'accel_y', 'accel_z', 'gyro_x', 'gyro_y', 'gyro_z']
        df_input = df[['accel_x', 'accel_y', 'accel_z', 'gyro_x', 'gyro_y', 'gyro_z']]
        df_input = df_input.astype('float32')
        df_output = df[['status']]
        df_output = df_output.astype('int')
        df_frames = df[['frame']]
        df_frames = df_frames.astype('int')

        input_data = np.append(input_data, df_input, axis=0)
        output_data = np.append(output_data, df_output, axis=0)

    input_data = input_data.reshape(NUM_SAMPLES, NUM_FRAMES, NUM_FEATURES)
    output_data = output_data.reshape(NUM_SAMPLES, NUM_FRAMES, 1)

    return input_data, output_data

input_data, output_data = read_files(DIR)

print(input_data.shape)
print(output_data.shape)

(2662, 1000, 6)
(2662, 1000, 1)


In [16]:
from keras.utils import to_categorical

# one-hot encoding
output_data = to_categorical(output_data)

from sklearn.model_selection import train_test_split

# split data into train and remaining
X_train, X_remaining, y_train, y_remaining = train_test_split(input_data, output_data, test_size=0.4, random_state=42)

# split the remaining data into test set and validation set (equal split)
X_test, X_val, y_test, y_val = train_test_split(X_remaining, y_remaining, test_size=0.5, random_state=42)

# build model
from keras.models import Sequential
from keras.layers import GRU, Dropout, Dense

model = Sequential([
    GRU(64, input_shape=(NUM_FRAMES, NUM_FEATURES), return_sequences=True),
    Dropout(0.2),
    GRU(64, return_sequences=True),
    Dropout(0.2),
    Dense(5, activation='sigmoid')
])

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.summary()

# train model
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_val, y_val))

# evaluate model
score = model.evaluate(X_test, y_test, batch_size=32)
print(score)

# save model

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 gru_4 (GRU)                 (None, 1000, 64)          13824     
                                                                 
 dropout_4 (Dropout)         (None, 1000, 64)          0         
                                                                 
 gru_5 (GRU)                 (None, 1000, 64)          24960     
                                                                 
 dropout_5 (Dropout)         (None, 1000, 64)          0         
                                                                 
 dense_2 (Dense)             (None, 1000, 5)           325       
                                                                 
Total params: 39109 (152.77 KB)
Trainable params: 39109 (152.77 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Epoch 1/10
50/50

In [17]:

model.save('./data/Model/RNN_LSTM_relu.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
